In [ ]:
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil
import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from ROOT import larocv

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pcacross = larocv.PCACrossing()
logger=pcacross.logger()
logger.set(0)
#set mask on
print "Logger level is...",logger.level()

In [ ]:
ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

In [ ]:
# make some lines
line1 = geo2d.Line("float")(50,50,-45)
line2 = geo2d.Line("float")(50,50,45)


ctor.clear()

    
pt_x=[]
pt_y=[]
ctor.push_back(ROOT.cv.Point_("int")(100,96))
ctor.push_back(ROOT.cv.Point_("int")(150,94))
ctor.push_back(ROOT.cv.Point_("int")(200,93))
ctor.push_back(ROOT.cv.Point_("int")(220,81))
ctor.push_back(ROOT.cv.Point_("int")(200,82))
ctor.push_back(ROOT.cv.Point_("int")(190,82))
ctor.push_back(ROOT.cv.Point_("int")(170,83))
ctor.push_back(ROOT.cv.Point_("int")(130,84))
ctor.push_back(ROOT.cv.Point_("int")(100,85))
ctor.push_back(ROOT.cv.Point_("int")(75,86))
ctor.push_back(ROOT.cv.Point_("int")(50,87))
ctor.push_back(ROOT.cv.Point_("int")(25,88))

ctor.push_back(ctor[0])
for pt in ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

plt.grid()
plt.show()

In [ ]:
pcacross

In [ ]:
# make a fake image...
a=np.zeros((450,450))

In [ ]:
# make a path representing this contour
path_list1 = []
path_list2 = []
path_list3 = []
for pt in ctor:
    path_list1.append((pt.x,pt.y)) 
    path_list2.append((pt.x,pt.y+100))
    path_list3.append((pt.x+100,pt.y))

In [ ]:
path1 = matplotlib.path.Path(path_list1)
path2 = matplotlib.path.Path(path_list2)
path3 = matplotlib.path.Path(path_list3)

In [ ]:
img1 = path1.contains_points(np.row_stack(np.where(a==0)).T).reshape(450,450).astype(np.uint8)
img2 = path2.contains_points(np.row_stack(np.where(a==0)).T).reshape(450,450).astype(np.uint8)
img3 = path3.contains_points(np.row_stack(np.where(a==0)).T).reshape(450,450).astype(np.uint8)

#set img1 values to be 20
img1=np.where(img1>0,img1+20,0)
#set img1 values to be 40
img2=np.where(img2>0,img2+40,0)
#set img2 values to be 60
img3=np.where(img3>0,img3+60,0)

#rotate 3
M = cv2.getRotationMatrix2D((450/2,450/2),45,1)
img3 = cv2.warpAffine(img3,M,(450,450))

#add them...
img = img1+img2+img3

plt.imshow(img,cmap='Greys',interpolation='none')
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.show()

In [ ]:
im2, contours, hierarchy = cv2.findContours(img.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
ctor_arr=np.array(contours)

In [ ]:
plt.imshow(img,cmap='Greys',interpolation='none')

for ctor in ctor_arr:
    ctor=ctor[:,0,:]
    plt.plot(ctor[:,0],ctor[:,1],'-',lw=2)
    
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
cluscomp=larocv.data.ClusterCompound()
for ctor in ctor_arr:
    ctor=ctor[:,0,:]
    atomic = larocv.data.AtomicContour()
    for pt in ctor:
        pt_cv = ROOT.cv.Point_('int')()
        pt_cv.x = pt[1]
        pt_cv.y = pt[0]
        atomic.push_back(pt_cv)
    print "This atomic size: ",atomic.size()
    cluscomp.push_back(atomic)

In [ ]:
#Got a mat object to send...
mat = larcv.as_gray_mat(larcv.as_image2d(img))

In [ ]:
intersections = pcacross.ComputeIntersections(cluscomp,mat)

In [ ]:
plt.imshow(img,cmap='Greys',interpolation='none')

for ctor in ctor_arr:
    ctor=ctor[:,0,:]
    plt.plot(ctor[:,0],ctor[:,1],'-',lw=2)

for intersection in intersections:
    plt.plot(intersection.y,
             intersection.x,'*',color='yellow',markersize=20)
    
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()